**Этот блокнот — упражнение из курса [Time Series](https://www.kaggle.com/learn/time-series). Учебник к нему можно открыть [по этой ссылке](https://www.kaggle.com/ryanholbrook/hybrid-models).**

---


# Введение #

Запустите эту ячейку, чтобы все настроить!

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.deterministic import DeterministicProcess
from xgboost import XGBRegressor


comp_dir = Path('../StoreSalesData/')
data_dir = Path("../dataset/")

store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

family_sales = (
    store_sales
    .groupby(['family', 'date'])
    .mean()
    .unstack('family')
    .loc['2017']
)

/var/folders/7k/q1n29k35429d2mnrn_3mbtjr0000gn/T/ipykernel_22840/3442574194.py:14: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  store_sales = pd.read_csv(
/var/folders/7k/q1n29k35429d2mnrn_3mbtjr0000gn/T/ipykernel_22840/3442574194.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  store_sales


-------------------------------------------------------------------------------

В следующих двух вопросах вы создадите бустинговый гибрид для датасета *Store Sales*, реализовав новый Python‑класс. Запустите эту ячейку, чтобы создать первоначальное определение класса. Вы добавите методы `fit` и `predict`, чтобы придать ему интерфейс, похожий на scikit‑learn.

In [2]:
# You'll add fit and predict methods to this minimal class
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None  # store column names from fit method


# 1) Определите метод fit для бустингового гибрида

Завершите определение `fit` для класса `BoostedHybrid`. При необходимости вернитесь к шагам 1 и 2 из раздела **Hybrid Forecasting with Residuals** в учебнике.

In [3]:
def fit(self, X_1, X_2, y):
    # YOUR CODE HERE: fit self.model_1
    self.model_1.fit(X_1, y)

    y_fit = pd.DataFrame(
        # YOUR CODE HERE: make predictions with self.model_1
        self.model_1.predict(X_1),
        index=X_1.index, columns=y.columns,
    )

    # YOUR CODE HERE: compute residuals
    y_resid = y - y_fit
    y_resid = y_resid.stack().squeeze() # wide to long

    # YOUR CODE HERE: fit self.model_2 on residuals
    self.model_2.fit(X_2, y_resid)

    # Save column names for predict method
    self.y_columns = y.columns
    # Save data for question checking
    self.y_fit = y_fit
    self.y_resid = y_resid


# Add method to class
BoostedHybrid.fit = fit

In [ ]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

-------------------------------------------------------------------------------

# 2) Определите метод predict для бустингового гибрида

Теперь определите метод `predict` для класса `BoostedHybrid`. При необходимости вернитесь к шагу 3 из раздела **Hybrid Forecasting with Residuals** в учебнике.

In [4]:
def predict(self, X_1, X_2):
    y_pred = pd.DataFrame(
        # YOUR CODE HERE: predict with self.model_1
        self.model_1.predict(X_1),
        index=X_1.index, columns=self.y_columns,
    )
    y_pred = y_pred.stack().squeeze()  # wide to long

    # YOUR CODE HERE: add self.model_2 predictions to y_pred
    y_pred += self.model_2.predict(X_2)
    
    return y_pred.unstack()  # long to wide


# Add method to class
BoostedHybrid.predict = predict

In [ ]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

-------------------------------------------------------------------------------

Теперь вы готовы использовать ваш новый класс `BoostedHybrid`, чтобы создать модель для данных *Store Sales*. Запустите следующую ячейку, чтобы подготовить данные для обучения.

In [5]:
# Target series
y = family_sales.loc[:, 'sales']


# X_1: Features for Linear Regression
dp = DeterministicProcess(index=y.index, order=1)
X_1 = dp.in_sample()


# X_2: Features for XGBoost
X_2 = family_sales.drop('sales', axis=1).stack()  # onpromotion feature

# Label encoding for 'family'
le = LabelEncoder()  # from sklearn.preprocessing
X_2 = X_2.reset_index('family')
X_2['family'] = le.fit_transform(X_2['family'])

# Label encoding for seasonality
X_2["day"] = X_2.index.day  # values are day of the month

/var/folders/7k/q1n29k35429d2mnrn_3mbtjr0000gn/T/ipykernel_22840/3489191612.py:11: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  X_2 = family_sales.drop('sales', axis=1).stack()  # onpromotion feature


# 3) Обучите бустинговый гибрид

Создайте гибридную модель, инициализировав класс `BoostedHybrid` с экземплярами `LinearRegression()` и `XGBRegressor()`.

In [6]:
# YOUR CODE HERE: Create LinearRegression + XGBRegressor hybrid with BoostedHybrid
model = BoostedHybrid(
    model_1=LinearRegression(),
    model_2=XGBRegressor()
)

# YOUR CODE HERE: Fit and predict
model.fit(X_1, X_2, y)
y_pred = model.predict(X_1, X_2)

y_pred = y_pred.clip(0.0)

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

-------------------------------------------------------------------------------

В зависимости от задачи вам могут подойти другие гибридные комбинации, чем линейная регрессия + XGBoost, которую вы создали в предыдущих вопросах. Запустите следующую ячейку, чтобы попробовать другие алгоритмы из scikit‑learn.

In [7]:
# Model 1 (trend)
from sklearn.linear_model import ElasticNet, Lasso, Ridge

# Model 2
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Boosted Hybrid

# YOUR CODE HERE: Try different combinations of the algorithms above
model = BoostedHybrid(
    model_1=Ridge(),
    model_2=KNeighborsRegressor(),
)

Это лишь некоторые предложения. Возможно, вы найдёте другие алгоритмы, которые вам понравятся, в [руководстве пользователя](https://scikit-learn.org/stable/supervised_learning.html) scikit‑learn.

Используйте код в этой ячейке, чтобы увидеть прогнозы, которые делает ваш гибрид.

In [8]:
y_train, y_valid = y[:"2017-07-01"], y["2017-07-02":]
X1_train, X1_valid = X_1[: "2017-07-01"], X_1["2017-07-02" :]
X2_train, X2_valid = X_2.loc[:"2017-07-01"], X_2.loc["2017-07-02":]

# Some of the algorithms above do best with certain kinds of
# preprocessing on the features (like standardization), but this is
# just a demo.
model.fit(X1_train, X2_train, y_train)
y_fit = model.predict(X1_train, X2_train).clip(0.0)
y_pred = model.predict(X1_valid, X2_valid).clip(0.0)

families = y.columns[0:6]
axs = y.loc(axis=1)[families].plot(
    subplots=True, sharex=True, figsize=(11, 9), **plot_params, alpha=0.5,
)
_ = y_fit.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C0', ax=axs)
_ = y_pred.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C3', ax=axs)
for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(family)

/Users/flexonafft/KaggleLearning/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/flexonafft/KaggleLearning/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/flexonafft/KaggleLearning/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


NameError: name 'plot_params' is not defined

# 4) Обучите с другими алгоритмами обучения

Когда будете готовы перейти дальше, запустите следующую ячейку для зачёта по этому вопросу.

In [ ]:
# View the solution (Run this cell to receive credit!)
q_4.check()

# Продолжим #

[**Преобразуйте любую задачу прогнозирования**](https://www.kaggle.com/ryanholbrook/forecasting-with-machine-learning) в задачу машинного обучения с четырьмя стратегиями ML‑прогнозирования.

---




*Есть вопросы или комментарии? Посетите [форум обсуждений курса](https://www.kaggle.com/learn/time-series/discussion), чтобы пообщаться с другими учащимися.*